In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

# Initialiser Faker pour générer des noms
fake = Faker('fr_FR')
np.random.seed(42)
random.seed(42)

# Paramètres
N_OBSERVATIONS = 200
MEAN_CHEQUES = 5
STD_CHEQUES = 2

# Listes de référence
REGIONS = ['Adamaoua', 'Nord-Ouest', 'Sud', 'Sud-Ouest']
DEPARTEMENTS = ['Mfoundi', 'Nyong-et-Kéllé', 'Wouri', 'Fako', 'Mezam', 'Bamboutos', 'Dja-et-Lobo']
FONCTIONS = {1: 'DIRECTEUR/CHEF DE L\'ÉTABLISSEMENT', 2: 'SAR', 6: 'AUTRE'}
CATEGORIES_FOSA = {
    1: 'Hôpital Général',
    2: 'Hôpital Central ou équivalent',
    3: 'Hôpital Régional',
    4: 'Hôpital de District ou équivalent',
    5: 'Centre Médical d\'Arrondissement',
    6: 'Centre de Santé Intégré',
    7: 'Dispensaire/Centre de Santé',
    8: 'Autre'
}
TYPES_GESTIONNAIRE = {1: 'ÉTAT', 2: 'PRIVE LAÏQUE', 3: 'PRIVE CONFESSIONNEL', 4: 'ONG/ASSOCIATION', 6: 'Autre'}
NIVEAUX_FOSA = {1: 'SONUC', 2: 'SONUB'}
STATUTS_FACTURE = {1: 'Rejetée', 2: 'Acceptée'}
MOTIFS_REJET = [
    'Documents incomplets',
    'Montant incorrect',
    'Signature manquante',
    'Prestation non conforme',
    'Bénéficiaire non éligible',
    'Date expirée',
    'Duplicata',
    ''
]
MOTIFS_REFERENCEMENT = [
    'Complications obstétricales',
    'Césarienne nécessaire',
    'Hémorragie post-partum',
    'Pré-éclampsie sévère',
    'Infection grave',
    'Détresse fœtale',
    'Dystocie',
    'Autres complications'
]

def generer_telephone():
    """Générer un numéro de téléphone camerounais"""
    prefixes = ['6', '2']
    prefix = random.choice(prefixes)
    if prefix == '6':
        second = random.choice(['5', '7', '8', '9'])
        numero = f"6{second}" + ''.join([str(random.randint(0, 9)) for _ in range(7)])
    else:
        numero = "2" + ''.join([str(random.randint(0, 9)) for _ in range(8)])
    return numero

def generer_date_enquete():
    """Générer une date aléatoire en 2025"""
    start_date = datetime(2025, 1, 1)
    end_date = datetime(2025, 10, 12)
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    return (start_date + timedelta(days=random_days)).strftime('%d/%m/2025')

# Générer les données principales
data_main = []

for i in range(N_OBSERVATIONS):
    # Section 00 - Identification
    region_code = f"{random.randint(1, 10):02d}"
    dept_code = f"{random.randint(1, 58):02d}"
    arr_code = f"{random.randint(1, 360):03d}"
    
    observation = {
        'F00Q01_nom_etablissement': f"ETABLISSEMENT_{i+1:03d}",
        'F00Q02_numero_sequentiel': f"{random.randint(1000, 9999)}",
        'F00Q04_region': region_code,
        'F00Q04_region_nom': random.choice(REGIONS),
        'F00Q05_departement': dept_code,
        'F00Q06_arrondissement': arr_code,
        'F00Q07_zone_implantation': random.choice([1, 2]),
        'F00Q08_evaluateur': f"EVAL_{random.randint(100, 999)}",
        'F00Q09_superviseur': f"SUPER_{random.randint(100, 999)}",
        'F00Q10_date_enquete': generer_date_enquete(),
        'F01Q01A_heure_debut': f"{random.randint(8, 16):02d}:{random.randint(0, 59):02d}",
        
        # Section 01 - Répondant
        'F01Q01_nom_repondant': fake.name(),
        'F01Q02_fonction_repondant': random.choice([1, 2, 6]),
        'F01Q03_tel_repondant': generer_telephone(),
        'F01Q04_tel_etablissement': generer_telephone(),
        
        # Section 02 - Type établissement
        'F02Q01_categorie_fosa': random.choice(list(CATEGORIES_FOSA.keys())),
        'F02Q02_type_gestionnaire': random.choice(list(TYPES_GESTIONNAIRE.keys())),
        'F02Q03_niveau_fosa': random.choice([1, 2]),
        
        # Section 03 - Facture
        'existence_facture': 1,  # Tous ont des factures
        'annee_facture': 2025,
        'mois_facture': random.randint(1, 10),
        'numero_facture': f"FACT_{random.randint(10000, 99999)}",
        'statut_facture_mc': random.choice([1, 2]),
        'categorie_prestation': random.randint(1, 5),
    }
    
    data_main.append(observation)

df_main = pd.DataFrame(data_main)

# Générer les données des chèques (table détaillée)
data_cheques = []

for idx, row in df_main.iterrows():
    # Nombre de chèques pour cette observation (distribution normale tronquée)
    n_cheques = int(np.random.normal(MEAN_CHEQUES, STD_CHEQUES))
    n_cheques = max(1, min(n_cheques, 20))  # Entre 1 et 20
    
    # Mettre à jour le nombre de chèques dans la table principale
    df_main.at[idx, 'nombre_cheques'] = n_cheques
    
    for j in range(n_cheques):
        cheque_valide = random.choices([1, 0], weights=[0.85, 0.15])[0]  # 85% validés
        patient_reference = random.choices([1, 0], weights=[0.20, 0.80])[0]  # 20% référencés
        
        cheque = {
            'id_observation': idx,
            'numero_facture': row['numero_facture'],
            'numero_cheque_ordre': j + 1,
            'numero_cheque': f"CHQ{random.randint(100000, 999999)}",
            'nom_beneficiaire': fake.name(),
            'valeur_mensuelle_fcfa': random.randint(5000, 250000),
            'numero_beneficiaire': f"BEN{random.randint(10000, 99999)}",
            'cheque_valide': cheque_valide,
            'motif_rejet': '' if cheque_valide else random.choice(MOTIFS_REJET[:-1]),
            'patient_reference': patient_reference,
            'numero_referencement': f"REF{random.randint(1000, 9999)}" if patient_reference else '',
            'motif_referencement': random.choice(MOTIFS_REFERENCEMENT) if patient_reference else ''
        }
        
        data_cheques.append(cheque)

df_cheques = pd.DataFrame(data_cheques)

# Statistiques
print("="*70)
print("STATISTIQUES DE LA BASE DE DONNÉES SIMULÉE")
print("="*70)
print(f"\nNombre total d'observations (établissements): {len(df_main)}")
print(f"Nombre total de chèques: {len(df_cheques)}")
print(f"\nStatistiques sur les chèques par établissement:")
print(f"  - Moyenne: {df_main['nombre_cheques'].mean():.2f}")
print(f"  - Écart-type: {df_main['nombre_cheques'].std():.2f}")
print(f"  - Min: {df_main['nombre_cheques'].min()}")
print(f"  - Max: {df_main['nombre_cheques'].max()}")
print(f"\nTaux de validation des chèques: {(df_cheques['cheque_valide'].sum() / len(df_cheques) * 100):.1f}%")
print(f"Taux de référencement des patients: {(df_cheques['patient_reference'].sum() / len(df_cheques) * 100):.1f}%")

# Sauvegarder dans Excel
output_file = 'base_donnees_fosa_simulation.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Feuille principale
    df_main.to_excel(writer, sheet_name='Données_Principales', index=False)
    
    # Feuille des chèques
    df_cheques.to_excel(writer, sheet_name='Détails_Chèques', index=False)
    
    # Feuille de statistiques
    stats = pd.DataFrame({
        'Indicateur': [
            'Nombre d\'observations',
            'Nombre de chèques',
            'Moyenne chèques/établissement',
            'Écart-type chèques/établissement',
            'Taux validation chèques',
            'Taux référencement patients'
        ],
        'Valeur': [
            len(df_main),
            len(df_cheques),
            df_main['nombre_cheques'].mean(),
            df_main['nombre_cheques'].std(),
            df_cheques['cheque_valide'].sum() / len(df_cheques) * 100,
            df_cheques['patient_reference'].sum() / len(df_cheques) * 100
        ]
    })
    stats.to_excel(writer, sheet_name='Statistiques', index=False)



STATISTIQUES DE LA BASE DE DONNÉES SIMULÉE

Nombre total d'observations (établissements): 200
Nombre total de chèques: 884

Statistiques sur les chèques par établissement:
  - Moyenne: 4.42
  - Écart-type: 1.82
  - Min: 1.0
  - Max: 10.0

Taux de validation des chèques: 85.1%
Taux de référencement des patients: 19.1%


In [2]:
print(f"\n{'='*70}")
print(f"Fichier Excel créé: {output_file}")
print(f"{'='*70}")
print("\nAperçu des données principales (5 premières lignes):")
print(df_main.head())
print("\nAperçu des chèques (10 premières lignes):")
print(df_cheques.head(10))


Fichier Excel créé: base_donnees_fosa_simulation.xlsx

Aperçu des données principales (5 premières lignes):
  F00Q01_nom_etablissement F00Q02_numero_sequentiel F00Q04_region  \
0        ETABLISSEMENT_001                     5012            02   
1        ETABLISSEMENT_002                     6635            07   
2        ETABLISSEMENT_003                     3677            03   
3        ETABLISSEMENT_004                     2489            03   
4        ETABLISSEMENT_005                     9837            06   

  F00Q04_region_nom F00Q05_departement F00Q06_arrondissement  \
0        Nord-Ouest                 02                   141   
1               Sud                 07                   184   
2         Sud-Ouest                 35                   126   
3          Adamaoua                 33                   253   
4          Adamaoua                 49                   083   

   F00Q07_zone_implantation F00Q08_evaluateur F00Q09_superviseur  \
0                      

In [3]:
import pandas as pd
import random

# Définir la seed pour reproductibilité
random.seed(42)

# Structure administrative réelle du Cameroun pour les 4 régions
REGIONS_DATA = {
    'Sud': {
        'Dja-et-Lobo': ['Bengbis', 'Djoum', 'Mintom', 'Oveng', 'Sangmélima'],
        'Mvila': ['Biwong-Bané', 'Biwong-Bulu', 'Ebolowa 1er', 'Ebolowa 2ème', 'Efoulan', 'Mengong', 'Mvangane', 'Ngoulemakong'],
        'Océan': ['Akom II', 'Campo', 'Kribi 1er', 'Kribi 2ème', 'Lolodorf', 'Mvengue', 'Niété'],
        'Vallée-du-Ntem': ['Ambam', 'Ma\'an', 'Olamze', 'Kyé-Ossi']
    },
    'Adamaoua': {
        'Djerem': ['Tibati', 'Meïganga', 'Dir', 'Ngaoundal'],
        'Faro-et-Déo': ['Galim-Tignère', 'Kontcha', 'Mayo-Baléo', 'Tignère'],
        'Mayo-Banyo': ['Bankim', 'Banyo', 'Mayo-Darle'],
        'Mbéré': ['Djohong', 'Meïganga', 'Dir'],
        'Vina': ['Belel', 'Martap', 'Mbé', 'Nganha', 'Ngaoundéré 1er', 'Ngaoundéré 2ème', 'Ngaoundéré 3ème']
    },
    'Nord-Ouest': {
        'Boyo': ['Belo', 'Fundong', 'Njinikom'],
        'Bui': ['Elak-Oku', 'Jakiri', 'Kumbo East', 'Kumbo West', 'Mbiame', 'Nkum', 'Oku'],
        'Donga-Mantung': ['Ako', 'Misaje', 'Ndu', 'Nkambe', 'Nwa'],
        'Menchum': ['Benakuma', 'Furu-Awa', 'Weh', 'Wum'],
        'Mezam': ['Bafut', 'Bamenda 1er', 'Bamenda 2ème', 'Bamenda 3ème', 'Santa', 'Tubah'],
        'Momo': ['Andek', 'Batibo', 'Mbengwi', 'Njikwa', 'Widikum'],
        'Ngo-Ketunjia': ['Babessi', 'Balikumbat', 'Jakiri', 'Ndop']
    },
    'Sud-Ouest': {
        'Fako': ['Buea', 'Limbe 1er', 'Limbe 2ème', 'Limbe 3ème', 'Muyuka', 'Tiko', 'West Coast'],
        'Koupé-Manengouba': ['Bangem', 'Nguti', 'Tombel'],
        'Lebialem': ['Alou', 'Fontem', 'Menji', 'Wabane'],
        'Manyu': ['Akwaya', 'Eyumojock', 'Mamfe Central', 'Upper Bayang'],
        'Meme': ['Kumba 1er', 'Kumba 2ème', 'Kumba 3ème', 'Mbonge', 'Konye'],
        'Ndian': ['Bamusso', 'Dikome Balue', 'Ekondo Titi', 'Idabato', 'Isangele', 'Kombo Abedimo', 'Mundemba', 'Toko']
    }
}

# Catégories de formations sanitaires
CATEGORIES = {
    1: 'Hôpital Général',
    2: 'Hôpital Central ou équivalent',
    3: 'Hôpital Régional',
    4: 'Hôpital de District',
    5: 'Centre Médical d\'Arrondissement',
    6: 'Centre de Santé Intégré',
    7: 'Dispensaire/Centre de Santé',
    8: 'Autre'
}

# Poids pour la distribution des catégories (les dispensaires et CSI sont plus nombreux)
POIDS_CATEGORIES = {
    1: 2,   # Hôpital Général - rare
    2: 1,   # Hôpital Central - très rare
    3: 3,   # Hôpital Régional - un par région
    4: 8,   # Hôpital de District - plusieurs
    5: 20,  # Centre Médical d'Arrondissement - nombreux
    6: 35,  # Centre de Santé Intégré - très nombreux
    7: 30,  # Dispensaire - très nombreux
    8: 1    # Autre - rare
}

def generer_fosa():
    """Générer une formation sanitaire aléatoire"""
    # Choisir une région
    region = random.choice(list(REGIONS_DATA.keys()))
    
    # Choisir un département dans cette région
    departement = random.choice(list(REGIONS_DATA[region].keys()))
    
    # Choisir un arrondissement dans ce département
    arrondissement = random.choice(REGIONS_DATA[region][departement])
    
    # Choisir une catégorie avec pondération
    categories = list(POIDS_CATEGORIES.keys())
    poids = list(POIDS_CATEGORIES.values())
    categorie_code = random.choices(categories, weights=poids)[0]
    categorie_nom = CATEGORIES[categorie_code]
    
    return {
        'Région': region,
        'Département': departement,
        'Arrondissement': arrondissement,
        'Catégorie_Code': categorie_code,
        'Catégorie': categorie_nom
    }

# Générer 200 formations sanitaires
print("Génération de 200 formations sanitaires...")
fosa_list = []

for i in range(200):
    fosa = generer_fosa()
    fosa['ID'] = f"FOSA_{i+1:03d}"
    fosa['Nom_Etablissement'] = f"{fosa['Catégorie']} - {fosa['Arrondissement']}"
    fosa_list.append(fosa)

# Créer le DataFrame
df = pd.DataFrame(fosa_list)

# Réorganiser les colonnes
df = df[['ID', 'Nom_Etablissement', 'Région', 'Département', 'Arrondissement', 'Catégorie_Code', 'Catégorie']]

# Afficher les statistiques
print("\n" + "="*80)
print("STATISTIQUES DE RÉPARTITION")
print("="*80)

print("\n📍 RÉPARTITION PAR RÉGION:")
print(df['Région'].value_counts().sort_index())

print("\n🏥 RÉPARTITION PAR CATÉGORIE:")
cat_counts = df['Catégorie'].value_counts()
for cat, count in cat_counts.items():
    print(f"  {cat}: {count} ({count/len(df)*100:.1f}%)")

print("\n📊 RÉPARTITION PAR DÉPARTEMENT (Top 10):")
print(df['Département'].value_counts().head(10))

print("\n" + "="*80)
print(f"TOTAL: {len(df)} formations sanitaires")
print("="*80)

# Sauvegarder dans Excel
output_file = 'formations_sanitaires_4_regions.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Feuille principale
    df.to_excel(writer, sheet_name='Formations_Sanitaires', index=False)
    
    # Feuille de statistiques par région
    stats_region = df.groupby(['Région', 'Catégorie']).size().reset_index(name='Nombre')
    stats_region_pivot = stats_region.pivot(index='Catégorie', columns='Région', values='Nombre').fillna(0)
    stats_region_pivot.to_excel(writer, sheet_name='Stats_Région_Catégorie')
    
    # Feuille de statistiques par département
    stats_dept = df.groupby(['Région', 'Département']).size().reset_index(name='Nombre')
    stats_dept.to_excel(writer, sheet_name='Stats_Département', index=False)
    
    # Feuille avec liste des arrondissements
    arr_list = df.groupby(['Région', 'Département', 'Arrondissement']).size().reset_index(name='Nombre_FOSA')
    arr_list.to_excel(writer, sheet_name='Liste_Arrondissements', index=False)

print(f"\n✅ Fichier Excel créé: {output_file}")

# Afficher un aperçu du tableau
print("\n📋 APERÇU DES DONNÉES (20 premières lignes):")
print(df.head(20).to_string(index=False))

# Créer aussi un CSV pour faciliter l'importation
csv_file = 'formations_sanitaires_4_regions.csv'
df.to_csv(csv_file, index=False, encoding='utf-8-sig')
print(f"\n✅ Fichier CSV créé: {csv_file}")

Génération de 200 formations sanitaires...

STATISTIQUES DE RÉPARTITION

📍 RÉPARTITION PAR RÉGION:
Région
Adamaoua      62
Nord-Ouest    52
Sud           43
Sud-Ouest     43
Name: count, dtype: int64

🏥 RÉPARTITION PAR CATÉGORIE:
  Dispensaire/Centre de Santé: 65 (32.5%)
  Centre de Santé Intégré: 59 (29.5%)
  Centre Médical d'Arrondissement: 46 (23.0%)
  Hôpital de District: 15 (7.5%)
  Hôpital Général: 6 (3.0%)
  Hôpital Régional: 5 (2.5%)
  Hôpital Central ou équivalent: 3 (1.5%)
  Autre: 1 (0.5%)

📊 RÉPARTITION PAR DÉPARTEMENT (Top 10):
Département
Dja-et-Lobo       15
Mayo-Banyo        14
Faro-et-Déo       13
Djerem            12
Vina              12
Mvila             11
Meme              11
Mbéré             11
Vallée-du-Ntem    10
Boyo               9
Name: count, dtype: int64

TOTAL: 200 formations sanitaires

✅ Fichier Excel créé: formations_sanitaires_4_regions.xlsx

📋 APERÇU DES DONNÉES (20 premières lignes):
      ID                                 Nom_Etablissement     Rég

## DICTIONNAIRE

In [3]:
import json
import re

def parse_dcf_file(dcf_file_path):
    """
    Parse un fichier .dcf CSPro et extrait les value sets.
    
    Args:
        dcf_file_path: Chemin vers le fichier .dcf
        
    Returns:
        dict: Un dictionnaire où les clés sont les noms des questions
              et les valeurs sont des dictionnaires {code: label}
    """
    with open(dcf_file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Le contenu du .dcf est en fait du JSON
    try:
        cspro_data = json.loads(content)
        return extract_value_sets_from_cspro(cspro_data)
    except json.JSONDecodeError:
        # Si ce n'est pas du JSON pur, essayer de le parser autrement
        print("Le fichier n'est pas en JSON standard, tentative de parsing alternatif...")
        return parse_dcf_text_format(content)


def parse_dcf_text_format(content):
    """
    Parse un fichier .dcf au format texte CSPro classique.
    
    Args:
        content: Contenu du fichier .dcf
        
    Returns:
        dict: Dictionnaire des value sets
    """
    result = {}
    
    # Pattern pour identifier les sections de variables
    current_var = None
    in_value_set = False
    
    lines = content.split('\n')
    
    for i, line in enumerate(lines):
        line = line.strip()
        
        # Détecter le début d'une variable
        if line.startswith('[Item]') or line.startswith('[Id]'):
            current_var = None
            in_value_set = False
            
        # Récupérer le nom de la variable
        if line.startswith('Name='):
            current_var = line.split('=', 1)[1].strip()
            
        # Détecter le début d'un ValueSet
        if line.startswith('[ValueSet]'):
            in_value_set = True
            if current_var and current_var not in result:
                result[current_var] = {}
                
        # Extraire les valeurs
        if in_value_set and current_var and '=' in line and not line.startswith('['):
            parts = line.split('=', 1)
            if len(parts) == 2:
                code = parts[0].strip()
                label = parts[1].strip().strip('"').strip("'")
                if code and label:
                    result[current_var][code] = label
    
    return result


def extract_value_sets_from_cspro(cspro_dict):
    """
    Transforme un dictionnaire CSPro JSON en dictionnaire de dictionnaires.
    
    Args:
        cspro_dict: Le dictionnaire CSPro chargé depuis le JSON
        
    Returns:
        dict: Un dictionnaire où les clés sont les noms des questions
              et les valeurs sont des dictionnaires {code: label}
    """
    result = {}
    
    # Parcourir les niveaux
    for level in cspro_dict.get('levels', []):
        # Parcourir les IDs du niveau
        for id_item in level.get('ids', {}).get('items', []):
            process_item(id_item, result)
        
        # Parcourir les enregistrements
        for record in level.get('records', []):
            # Parcourir les items de chaque enregistrement
            for item in record.get('items', []):
                process_item(item, result)
    
    return result


def process_item(item, result):
    """
    Traite un item et extrait ses valueSets s'ils existent.
    
    Args:
        item: L'item à traiter
        result: Le dictionnaire résultat à remplir
    """
    item_name = item.get('name')
    value_sets = item.get('valueSets', [])
    
    if item_name and value_sets:
        # Créer un dictionnaire pour cet item
        item_dict = {}
        
        # Parcourir tous les valueSets
        for value_set in value_sets:
            for value in value_set.get('values', []):
                # Extraire le code et le label
                pairs = value.get('pairs', [])
                labels = value.get('labels', [])
                
                if pairs and labels:
                    code = pairs[0].get('value')
                    label = labels[0].get('text')
                    
                    if code and label:
                        item_dict[code] = label
        
        # Ajouter au résultat si on a trouvé des valeurs
        if item_dict:
            result[item_name] = item_dict


    

In [4]:
# Parser le fichier .dcf
dcf_file = 'CHEQUE.dcf'  # Remplacez par le chemin de votre fichier
    
try:
        value_sets_dict = parse_dcf_file(dcf_file)
        
        # Afficher quelques statistiques
        print(f"Nombre de questions avec des modalités: {len(value_sets_dict)}")
        print(f"\nListe des questions trouvées:")
        for question in sorted(value_sets_dict.keys()):
            print(f"  - {question} ({len(value_sets_dict[question])} modalités)")
        
        # Afficher quelques exemples
        print("\n" + "="*50)
        print("Exemple pour S0Q01 (REGION):")
        if 'S0Q01' in value_sets_dict:
            for code, label in value_sets_dict['S0Q01'].items():
                print(f"  {code}: {label}")
        
        print("\n" + "="*50)
        print("Exemple pour MOIS:")
        if 'MOIS' in value_sets_dict:
            for code, label in value_sets_dict['MOIS'].items():
                print(f"  {code}: {label}")
        
        print("\n" + "="*50)
        print("Exemple pour STATUT:")
        if 'STATUT' in value_sets_dict:
            for code, label in value_sets_dict['STATUT'].items():
                print(f"  {code}: {label}")
        
        # Sauvegarder le résultat
        output_file = 'value_sets_output.json'
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(value_sets_dict, f, ensure_ascii=False, indent=2)
        
        print(f"\n✓ Dictionnaire sauvegardé dans '{output_file}'")
        
        # Optionnel : Sauvegarder aussi en Python
        output_py = 'value_sets_output.py'
        with open(output_py, 'w', encoding='utf-8') as f:
            f.write("# Dictionnaire des modalités CSPro\n\n")
            f.write("VALUE_SETS = ")
            f.write(json.dumps(value_sets_dict, ensure_ascii=False, indent=2))
        
        print(f"✓ Dictionnaire Python sauvegardé dans '{output_py}'")
        
except FileNotFoundError:
        print(f"Erreur: Le fichier '{dcf_file}' n'a pas été trouvé.")
except Exception as e:
        print(f"Erreur lors du traitement du fichier: {e}")
        import traceback
        traceback.print_exc()

Nombre de questions avec des modalités: 19

Liste des questions trouvées:
  - ACTION (1 modalités)
  - ACTION1 (1 modalités)
  - ANNEE (3 modalités)
  - CAT (3 modalités)
  - CHEQUE_ID (474 modalités)
  - F01Q02 (3 modalités)
  - F01Q06 (3 modalités)
  - FMALFERME (2 modalités)
  - MALFERME (2 modalités)
  - MOIS (12 modalités)
  - MOTIF (10 modalités)
  - S0Q01 (2 modalités)
  - S0Q02 (24 modalités)
  - S0Q03 (109 modalités)
  - S0Q3 (2 modalités)
  - STATCH (3 modalités)
  - STATINC (2 modalités)
  - STATUT (19 modalités)
  - TYPPRES (55 modalités)

Exemple pour S0Q01 (REGION):
  1: Adamaoua
  2: Sud

Exemple pour MOIS:
  1: Janvier
  2: Fervrier
  3: Mars
  4: Avril
  5: Mai
  6: Juin
  7: Juillet
  8: Aôut
  9: Septembre
  10: Octobre
  11: Novembre
  12: Decembre

Exemple pour STATUT:
  1: CPN1
  2: CPN2
  3: CPN3
  4: CPN4
  5: ECHOGRAPHIE
  6: ACCOUCHEMENT
  7: CESARIENNE
  8: CPONJ6
  9: CPON42
  10: CPNNEJ6
  11: CPNNEJ42
  12: COMPLICATION GROSSESSE
  13: COMPLICATION ACCOUCH